# **Skip List and Hash Table**

## Skip List

A Skip List is a probabilistic data structure that allows insertions, searches, and deletions with an expected performance of **O(log n)**.

It operates like a linked list with multiple levels. Each higher level allows larger “jumps,” speeding up access.

In this exercise, we implemented:
- Ordered insertion by book code.
- Search that prints the path traversed.
- Removal of elements from the structure.

The height of nodes is generated randomly, based on a probability (usually 0.5).


### Part A - Construction and Insertion of the books in the list

In [1]:
import random  # We will need random numbers to choose node heights

# Each node will have: key, value, and a list of pointers to the next nodes at each level
class Node:
    def __init__(self, key, value, level):
        self.key = key
        self.value = value
        self.forward = [None] * (level + 1)  # forward pointers for each level

# The famous Skip List
class SkipList:
    def __init__(self, max_level=3, p=0.5):
        self.max_level = max_level  # maximum level (like the list's ceiling)
        self.p = p  # probability for a level up (50% by default)
        self.header = Node(-1, None, max_level)  # header node (value -1 just to mark)
        self.level = 0  # starts with height 0, grows as items are inserted

    # Generates a random height for the new node (like flipping a coin to see if it goes up)
    def random_level(self):
        lvl = 0
        while random.random() < self.p and lvl < self.max_level:
            lvl += 1
        return lvl

    # Insertion in the skip list
    def insert(self, key, value):
        update = [None] * (self.max_level + 1)  # stores where we passed through
        current = self.header

        # Let's descend from the top to level 0 looking for where to fit
        for i in reversed(range(self.level + 1)):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current  # marks the last node before the insertion spot

        current = current.forward[0]  # reaches the base level

        # If the key does not exist yet, we add the new one
        if current is None or current.key != key:
            rlevel = self.random_level()
            if rlevel > self.level:
                for i in range(self.level + 1, rlevel + 1):
                    update[i] = self.header  # completes the new levels with the header
                self.level = rlevel  # updates the current level of the list

            new_node = Node(key, value, rlevel)
            # Connects the forward pointers in each level
            for i in range(rlevel + 1):
                new_node.forward[i] = update[i].forward[i]
                update[i].forward[i] = new_node

    # Search in the skip list, printing the path it traversed
    def search(self, key, verbose=True):
        current = self.header
        path = []

        for i in reversed(range(self.level + 1)):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            path.append(current.key)

        current = current.forward[0]

        if current and current.key == key:
            if verbose:
                print(f"Found: {key}")
                print(f"Path traversed: {path}")
            return current.value
        else:
            if verbose:
                print(f"{key} not found.")
                print(f"Path traversed: {path}")
            return None

    # Simple removal (no drama)
    def remove(self, key):
        update = [None] * (self.max_level + 1)
        current = self.header

        for i in reversed(range(self.level + 1)):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current

        current = current.forward[0]

        if current and current.key == key:
            for i in range(self.level + 1):
                if update[i].forward[i] != current:
                    continue
                update[i].forward[i] = current.forward[i]

            # Decreases the level of the skip list if the upper ones became empty
            while self.level > 0 and self.header.forward[self.level] is None:
                self.level -= 1

    # Show the entire Skip List from top to bottom
    def display(self):
        print("Skip List Levels:")
        for i in range(self.level, -1, -1):
            print(f"Level {i}: ", end='')
            node = self.header.forward[i]
            while node:
                print(node.key, end=' ')
                node = node.forward[i]
            print()

In [2]:
# Creating the list
skip_list = SkipList()

# Inserting books
books = [(101, "Algorithms and Data Structures"),(29, "Artificial Intelligence"),(203, "Advanced Machine Learning"),
(88, "Python Programming"),(175, "Neural Networks and Deep Learning"),(256, "Distributed Systems"),(191, "Computer Vision"),
(317, "Mathematical Logic"),(125, "Databases"),(297, "Operating Systems"),(410, "Computer Networks"),(281, "Compilers"),
(373, "Discrete Mathematics"),(182, "Digital Image Processing"),(299, "Algorithm Design and Analysis"),(437, "Software Engineering")]
for code, name in books:
    skip_list.insert(code, name)

skip_list.display()

Skip List Levels:
Level 3: 101 437 
Level 2: 101 437 
Level 1: 101 125 175 317 437 
Level 0: 29 88 101 125 175 182 191 203 256 281 297 299 317 373 410 437 


### Part B - Implement operations

In [3]:
# Inserting the book (408, "Natural Language Processing")

code = 408
name = "Natural Language Processing"

skip_list.insert(code, name)
skip_list.display()

Skip List Levels:
Level 3: 101 437 
Level 2: 101 437 
Level 1: 101 125 175 317 408 437 
Level 0: 29 88 101 125 175 182 191 203 256 281 297 299 317 373 408 410 437 


In [4]:
# Searching for the book (373, "Discrete Mathematics") and showing the path traversed.
skip_list.search(373)

Found: 373
Path traversed: [101, 101, 317, 317]


'Discrete Mathematics'

In [5]:
# Searching for the book (400, "Graph Theory") and showing the path traversed during the search.
skip_list.search(400)

400 not found.
Path traversed: [101, 101, 317, 373]


In [6]:
# Removing the book "Advanced Machine Learning" from the Skip List.
skip_list.remove(203)

skip_list.display()

Skip List Levels:
Level 3: 101 437 
Level 2: 101 437 
Level 1: 101 125 175 317 408 437 
Level 0: 29 88 101 125 175 182 191 256 281 297 299 317 373 408 410 437 


### Part C – Complexity Analysis

The dispersion factor `d` represents the **probability that an element from level `i` is replicated to level `i+1`**.

This means that when moving up a level, the average number of nodes **decreases by a factor of `d`**. Mathematically, we have:

| Level | Expected number of nodes |
|--------|---------------------------|
| 0      | n                         |
| 1      | n / d                     |
| 2      | n / d²                    |
| 3      | n / d³                    |
| ...    | ...                       |
| k      | n / dᵏ                    |

At the top level, we expect to have only **1 node**, that is:
\( n / d^k = 1 \)

This implies: \( n = d^k \Rightarrow k = \log_d(n) \)

Therefore, a Skip List with `n` elements will have at most **1 + log_d(n)** levels.

---

#### Average Number of Jumps

- For each node at level `i+1`, there are expected to be `d` nodes at level `i`.
- Thus, to find an element, an average of `d/2` jumps per level are needed before descending to the next level.

---

#### Average Search Time

Multiplying the average number of levels by the average number of jumps per level, we get:

\[
\text{Average search time} \approx (d / 2) \times (1 + \log_d(n)) = O(\log n)
\]

Therefore, since `d` is constant (for example `d = 2`), the average search time in a Skip List is **logarithmic**, i.e., **O(log n)**.






## Hash Table with Linear Probing

In this stage, we will implement a **hash table of size 17** with the function: `h(k) = k mod 17`.

Collision resolution will use **linear probing**, that is, if position `h(k)` is occupied, we try `h(k) + 1`, `h(k) + 2`, ..., until finding a free slot.

We will use the following data (secret code):  
25, 47, 98, 13, 52, 75, 67, 32, 81, 11, 89, 55, 29, 39, 42

The table should show:
- The calculated index
- The number of collisions resolved
- The final state of the table





### Part A – Building the Hash Table

Choosing `M` as a **prime number** helps distribute the elements more evenly in the table, especially when using the function `h(k) = k mod M`.

**Benefits of `M` being prime:**
- Ensures that the remainders `k mod M` are **more uniformly distributed**.
- Greatly reduces the risk of repeated collisions.

**Example:**

If `M = 16` (not prime) and `k` is a multiple of 4, then `k mod 16` will always be a multiple of 4 → poor distribution.

If `M = 17` (prime), `k mod 17` is more uniform even when `k` follows patterns.

This means that all elements are distributed in positions from `0` to `M - 1`.

In [7]:
class HashTableLinear:
    def __init__(self, size=17):
        self.size = size
        self.table = [None] * size
        self.collisions = 0

    def hash(self, key):
        return key % self.size

    def insert(self, key):
        index = self.hash(key)
        start_index = index
        probes = 0

        while self.table[index] is not None:
            probes += 1
            self.collisions += 1
            index = (index + 1) % self.size
            if index == start_index:
                raise Exception("Table full! Could not insert.")

        self.table[index] = key
        print(f"Inserted {key} at position {index} with {probes} probes.")

    def display(self):
        print("\nHash Table (Linear Probing):")
        for i, val in enumerate(self.table):
            print(f"[{i}] -> {val}")
        print(f"\nTotal collisions: {self.collisions}")

In [8]:
hash_table = HashTableLinear()

keys = [25, 47, 98, 13, 52, 75, 67, 32, 81, 11, 89, 55, 29, 39, 42]

for key in keys:
    hash_table.insert(key)

hash_table.display()

Inserted 25 at position 8 with 0 probes.
Inserted 47 at position 13 with 0 probes.
Inserted 98 at position 14 with 1 probes.
Inserted 13 at position 15 with 2 probes.
Inserted 52 at position 1 with 0 probes.
Inserted 75 at position 7 with 0 probes.
Inserted 67 at position 16 with 0 probes.
Inserted 32 at position 0 with 2 probes.
Inserted 81 at position 2 with 6 probes.
Inserted 11 at position 11 with 0 probes.
Inserted 89 at position 4 with 0 probes.
Inserted 55 at position 5 with 1 probes.
Inserted 29 at position 12 with 0 probes.
Inserted 39 at position 6 with 1 probes.
Inserted 42 at position 9 with 1 probes.

Hash Table (Linear Probing):
[0] -> 32
[1] -> 52
[2] -> 81
[3] -> None
[4] -> 89
[5] -> 55
[6] -> 39
[7] -> 75
[8] -> 25
[9] -> 42
[10] -> None
[11] -> 11
[12] -> 29
[13] -> 47
[14] -> 98
[15] -> 13
[16] -> 67

Total collisions: 14


### Part B – Double Hashing

In this method, we use two hash functions:

- **h₁(k) = k mod 17** → base position  
- **h₂(k) = 1 + (k mod 16)** → probing increment

When a collision occurs, we try:  
`h(k, i) = (h₁(k) + i * h₂(k)) mod 17`

This prevents consecutive linear probes and reduces the possibility of cluster formation.

We will insert the same values from Part A:  
25, 47, 98, 13, 52, 75, 67, 32, 81, 11, 89, 55, 29, 39, 42

In [9]:
class HashTableDoubleHashing:
    def __init__(self, size=17):
        self.size = size
        self.table = [None] * size
        self.collisions = 0

    def h1(self, key):
        return key % self.size

    def h2(self, key):
        return 1 + (key % (self.size - 1))  # prevents zero step

    def insert(self, key):
        i = 0
        while i < self.size:
            index = (self.h1(key) + i * self.h2(key)) % self.size
            if self.table[index] is None:
                self.table[index] = key
                print(f"Inserted {key} at position {index} with {i} probes.")
                return
            else:
                i += 1
                self.collisions += 1
        raise Exception("Table full! Could not insert.")

    def display(self):
        print("\nHash Table (Double Hashing):")
        for i, val in enumerate(self.table):
            print(f"[{i}] -> {val}")
        print(f"\nTotal collisions: {self.collisions}")

In [10]:
hash_table = HashTableDoubleHashing()

keys = [25, 47, 98, 13, 52, 75, 67, 32, 81, 11, 89, 55, 29, 39, 42]

for key in keys:
    hash_table.insert(key)

hash_table.display()

Inserted 25 at position 8 with 0 probes.
Inserted 47 at position 13 with 0 probes.
Inserted 98 at position 16 with 1 probes.
Inserted 13 at position 10 with 1 probes.
Inserted 52 at position 1 with 0 probes.
Inserted 75 at position 7 with 0 probes.
Inserted 67 at position 3 with 1 probes.
Inserted 32 at position 15 with 0 probes.
Inserted 81 at position 0 with 2 probes.
Inserted 11 at position 11 with 0 probes.
Inserted 89 at position 4 with 0 probes.
Inserted 55 at position 12 with 1 probes.
Inserted 29 at position 9 with 1 probes.
Inserted 39 at position 5 with 0 probes.
Inserted 42 at position 2 with 1 probes.

Hash Table (Double Hashing):
[0] -> 81
[1] -> 52
[2] -> 42
[3] -> 67
[4] -> 89
[5] -> 39
[6] -> None
[7] -> 75
[8] -> 25
[9] -> 29
[10] -> 13
[11] -> 11
[12] -> 55
[13] -> 47
[14] -> None
[15] -> 32
[16] -> 98

Total collisions: 8


### Part C – Dynamic Resizing

To ensure hash table efficiency even with many entries, we apply **dynamic resizing**.

**Resizing rules:**

1. **Monitor table occupancy:**  
   If the number of elements exceeds 70% of capacity, it is time to resize.

2. **New size M':**  
   Choose the next prime number greater than twice the original table size.  
   If M = 17, then M' = **37**

3. **Rehash:**  
   Recalculate `h(k) = k mod M'` for all previously inserted keys.

4. **New Insertions:**  
   After resizing, insert the remaining keys.

In [11]:
class HashTableLinearResizable:
    def __init__(self, size=17):
        self.size = size
        self.table = [None] * size
        self.count = 0
        self.collisions = 0

    def hash(self, key):
        return key % self.size

    def insert(self, key):
        index = self.hash(key)
        start_index = index
        probes = 0

        while self.table[index] is not None:
            probes += 1
            self.collisions += 1
            index = (index + 1) % self.size
            if index == start_index:
                raise Exception("Table full!")

        self.table[index] = key
        self.count += 1
        print(f"Inserted {key} at position {index} with {probes} probes.")

        # Checks if occupancy exceeded 70%
        if self.count / self.size > 0.7:
            self.resize()

    def resize(self):
        print("\nResizing: occupancy exceeded 70%")
        old_table = [key for key in self.table if key is not None]
        self.size = 37  # New prime size > 2 * 17
        self.table = [None] * self.size
        self.count = 0
        self.collisions = 0
        print("Re-hashing existing elements...\n")
        for key in old_table:
            self.insert(key)

    def display(self):
        print("\nHash Table:")
        for i, val in enumerate(self.table):
            print(f"[{i}] -> {val}")
        print(f"\nTotal collisions: {self.collisions}")

In [12]:
# Initializes table with size 17
hash_table = HashTableLinearResizable()

# Inserting initial data
initial_keys = [25, 47, 98, 13, 52, 75, 67, 32, 81, 11, 89, 55, 29, 39, 42]
for key in initial_keys:
    hash_table.insert(key)

# Inserting new keys after resizing
new_keys = [12, 23, 66, 91, 44, 56, 78, 99]
for key in new_keys:
    hash_table.insert(key)

hash_table.display()

Inserted 25 at position 8 with 0 probes.
Inserted 47 at position 13 with 0 probes.
Inserted 98 at position 14 with 1 probes.
Inserted 13 at position 15 with 2 probes.
Inserted 52 at position 1 with 0 probes.
Inserted 75 at position 7 with 0 probes.
Inserted 67 at position 16 with 0 probes.
Inserted 32 at position 0 with 2 probes.
Inserted 81 at position 2 with 6 probes.
Inserted 11 at position 11 with 0 probes.
Inserted 89 at position 4 with 0 probes.
Inserted 55 at position 5 with 1 probes.

Resizing: occupancy exceeded 70%
Re-hashing existing elements...

Inserted 32 at position 32 with 0 probes.
Inserted 52 at position 15 with 0 probes.
Inserted 81 at position 7 with 0 probes.
Inserted 89 at position 16 with 1 probes.
Inserted 55 at position 18 with 0 probes.
Inserted 75 at position 1 with 0 probes.
Inserted 25 at position 25 with 0 probes.
Inserted 11 at position 11 with 0 probes.
Inserted 47 at position 10 with 0 probes.
Inserted 98 at position 24 with 0 probes.
Inserted 13 at pos